# Generating Genome Feature Tracks

In this notebook, I'll use the [NCBI assembly](https://www.ncbi.nlm.nih.gov/assembly/247141) to create genome feature tracks for *Fundulus heteroclitus*

## 0. Set working directory and variables

In [14]:
!pwd

/Users/yaaminivenkataraman/Documents/killifish-hypoxia-RRBS/code


In [15]:
cd ..

/Users/yaaminivenkataraman/Documents/killifish-hypoxia-RRBS


In [17]:
!mkdir genome-features

In [18]:
cd genome-features

/Users/yaaminivenkataraman/Documents/killifish-hypoxia-RRBS/genome-features


In [6]:
!which bedtools

/opt/homebrew/bin/bedtools


In [7]:
bedtoolsDirectory = "/opt/homebrew/bin"

## 1. Download NCBI assembly

I downloaded the GFF from [this link](https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/011/125/445/GCF_011125445.2_MU-UCD_Fhet_4.1/).

In [19]:
!curl https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/011/125/445/GCF_011125445.2_MU-UCD_Fhet_4.1/GCF_011125445.2_MU-UCD_Fhet_4.1_genomic.gff.gz > GCF_011125445.2_MU-UCD_Fhet_4.1_genomic.gff.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.5M  100 16.5M    0     0  14.0M      0  0:00:01  0:00:01 --:--:-- 14.0M


In [31]:
!gunzip Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf.gz

In [33]:
!head -n100 Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf

#!genome-build Fundulus_heteroclitus-3.0.2
#!genome-version Fundulus_heteroclitus-3.0.2
#!genome-date 2015-01
#!genome-build-accession GCA_000826765.1
#!genebuild-last-updated 2018-07
KN805525.1	ensembl	gene	2500	10348	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805525.1	ensembl	transcript	2500	10348	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "cdx1a-201"; transcript_source "ensembl"; transcript_biotype "protein_coding";
KN805525.1	ensembl	exon	10201	10348	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "1"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "cdx1a-201"; transcript_source "ensembl"; transcript_biotype "protein_codin

## 2. Prepare for feature track creation

Before I pull out feature tracks, I need to know which databases were used for annotation, which features I can expect and how many of them there are, and identify chromosome lengths.

In [35]:
#Database identifiers for extracting features
!cut -f2 Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf | sort | uniq -c | tail

   1 #!genebuild-last-updated 2018-07
   1 #!genome-build Fundulus_heteroclitus-3.0.2
   1 #!genome-build-accession GCA_000826765.1
   1 #!genome-date 2015-01
   1 #!genome-version Fundulus_heteroclitus-3.0.2
 145 RefSeq
860089 ensembl


In [37]:
#Count the number of unique features in the GFF
!cut -f3 Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf | sort | uniq -c

   1 #!genebuild-last-updated 2018-07
   1 #!genome-build Fundulus_heteroclitus-3.0.2
   1 #!genome-build-accession GCA_000826765.1
   1 #!genome-date 2015-01
   1 #!genome-version Fundulus_heteroclitus-3.0.2
341549 CDS
   2 Selenocysteine
350439 exon
25788 five_prime_utr
23469 gene
30408 start_codon
34476 stop_codon
18506 three_prime_utr
35597 transcript


In [30]:
#Chromosome length information
!head mummichog.chrom.length

KN805525.1	6356336
KN805526.1	6250690
KN811289.1	6115441
KN811310.1	5246820
KN811405.1	5058772
KN811340.1	5018176
KN811371.1	4958163
KN811339.1	4925094
KN811372.1	4897111
KN811406.1	4652060


## 2. Generate genome feature tracks

I will extract CDS, exon, gene, lncRNA and mRNA tracks. I can then use those existing tracks to produce intron and  intergenic tracks, as well as 1 kb upstream and downstream flanking regions with `bedtools`. I will also use the RepeatMasker output from NCBI for my transposable element track.

In [27]:
!{bedtoolsDirectory}/bedtools --version

bedtools v2.30.0


### 2a. Gene

In [40]:
#Isolate gene entries from multiple annotation databses. Tab mus be included between database and feature
#Sort output for downstream use
#Include chromosome name information
!grep -e "RefSeq	gene" -e "ensembl	gene" \
Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf \
| {bedtoolsDirectory}/sortBed \
-faidx mummichog.chrom.length \
> Fundulus_heteroclitus-3.0.2.105-gene.gff

In [41]:
!head Fundulus_heteroclitus-3.0.2.105-gene.gff
!wc -l Fundulus_heteroclitus-3.0.2.105-gene.gff

KN805525.1	ensembl	gene	2500	10348	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805525.1	ensembl	gene	38959	57020	.	+	.	gene_id "ENSFHEG00000014326"; gene_version "1"; gene_name "pdgfrb"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805525.1	ensembl	gene	62194	78208	.	+	.	gene_id "ENSFHEG00000014282"; gene_version "1"; gene_name "csf1ra"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805525.1	ensembl	gene	85895	96387	.	-	.	gene_id "ENSFHEG00000014227"; gene_version "1"; gene_name "hmgxb3"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805525.1	ensembl	gene	96807	105515	.	+	.	gene_id "ENSFHEG00000014113"; gene_version "1"; gene_name "si:ch73-42p12.2"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805525.1	ensembl	gene	103755	106513	.	+	.	gene_id "ENSFHEG00000014098"; gene_version "1"; gene_name "SLC35A4"; gene_source "ensembl"; gene_biotype "protein_coding";
KN805

### 2b. CDS

In [42]:
!grep -e "RefSeq	CDS" -e "ensembl	CDS" \
Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf \
| {bedtoolsDirectory}/sortBed \
-faidx mummichog.chrom.length \
> Fundulus_heteroclitus-3.0.2.105-CDS.gff

In [43]:
!head Fundulus_heteroclitus-3.0.2.105-CDS.gff
!wc -l Fundulus_heteroclitus-3.0.2.105-CDS.gff

KN805525.1	ensembl	CDS	2503	2651	.	-	2	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "4"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "cdx1a-201"; transcript_source "ensembl"; transcript_biotype "protein_coding"; protein_id "ENSFHEP00000012884"; protein_version "1";
KN805525.1	ensembl	CDS	5166	5274	.	-	0	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "3"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "cdx1a-201"; transcript_source "ensembl"; transcript_biotype "protein_coding"; protein_id "ENSFHEP00000012884"; protein_version "1";
KN805525.1	ensembl	CDS	8335	8483	.	-	2	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "2"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_

### 2c. Exon

In [45]:
!grep -e "RefSeq	exon" -e "ensembl	exon" \
Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf \
| {bedtoolsDirectory}/sortBed \
-faidx mummichog.chrom.length \
> Fundulus_heteroclitus-3.0.2.105-exon.gff

In [46]:
!head Fundulus_heteroclitus-3.0.2.105-exon.gff
!wc -l Fundulus_heteroclitus-3.0.2.105-exon.gff

KN805525.1	ensembl	exon	2500	2651	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "4"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "cdx1a-201"; transcript_source "ensembl"; transcript_biotype "protein_coding"; exon_id "ENSFHEE00000192221"; exon_version "1";
KN805525.1	ensembl	exon	5166	5274	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "3"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "cdx1a-201"; transcript_source "ensembl"; transcript_biotype "protein_coding"; exon_id "ENSFHEE00000072033"; exon_version "1";
KN805525.1	ensembl	exon	8335	8483	.	-	.	gene_id "ENSFHEG00000014345"; gene_version "1"; transcript_id "ENSFHET00000020248"; transcript_version "1"; exon_number "2"; gene_name "cdx1a"; gene_source "ensembl"; gene_biotype "protein_coding"; 

### 2d. 5' UTR

In [47]:
!grep -e "RefSeq	five_prime_utr" -e "ensembl	five_prime_utr" \
Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf \
| {bedtoolsDirectory}/sortBed \
-faidx mummichog.chrom.length \
> Fundulus_heteroclitus-3.0.2.105-five_prime_utr.gff

In [48]:
!head Fundulus_heteroclitus-3.0.2.105-five_prime_utr.gff
!wc -l Fundulus_heteroclitus-3.0.2.105-five_prime_utr.gff

KN805525.1	ensembl	five_prime_utr	62194	62306	.	+	.	gene_id "ENSFHEG00000014282"; gene_version "1"; transcript_id "ENSFHET00000020149"; transcript_version "1"; gene_name "csf1ra"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "csf1ra-201"; transcript_source "ensembl"; transcript_biotype "protein_coding";
KN805525.1	ensembl	five_prime_utr	62562	62579	.	+	.	gene_id "ENSFHEG00000014282"; gene_version "1"; transcript_id "ENSFHET00000020149"; transcript_version "1"; gene_name "csf1ra"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "csf1ra-201"; transcript_source "ensembl"; transcript_biotype "protein_coding";
KN805525.1	ensembl	five_prime_utr	96807	96912	.	+	.	gene_id "ENSFHEG00000014113"; gene_version "1"; transcript_id "ENSFHET00000019976"; transcript_version "1"; gene_name "si:ch73-42p12.2"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "si:ch73-42p12.2-201"; transcript_source "ensembl"; transcript_biotype "protein_c

### 2e. 3' UTR

In [49]:
!grep -e "RefSeq	three_prime_utr" -e "ensembl	three_prime_utr" \
Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.105.gtf \
| {bedtoolsDirectory}/sortBed \
-faidx mummichog.chrom.length \
> Fundulus_heteroclitus-3.0.2.105-three_prime_utr.gff

In [50]:
!head Fundulus_heteroclitus-3.0.2.105-three_prime_utr.gff
!wc -l Fundulus_heteroclitus-3.0.2.105-three_prime_utr.gff

KN805525.1	ensembl	three_prime_utr	76711	78208	.	+	.	gene_id "ENSFHEG00000014282"; gene_version "1"; transcript_id "ENSFHET00000020149"; transcript_version "1"; gene_name "csf1ra"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "csf1ra-201"; transcript_source "ensembl"; transcript_biotype "protein_coding";
KN805525.1	ensembl	three_prime_utr	105041	105515	.	+	.	gene_id "ENSFHEG00000014113"; gene_version "1"; transcript_id "ENSFHET00000032071"; transcript_version "1"; gene_name "si:ch73-42p12.2"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "si:ch73-42p12.2-202"; transcript_source "ensembl"; transcript_biotype "protein_coding";
KN805525.1	ensembl	three_prime_utr	105041	105515	.	+	.	gene_id "ENSFHEG00000014113"; gene_version "1"; transcript_id "ENSFHET00000019954"; transcript_version "1"; gene_name "si:ch73-42p12.2"; gene_source "ensembl"; gene_biotype "protein_coding"; transcript_name "si:ch73-42p12.2-203"; transcript_source "ensembl"; tran

### 2f. Non-coding sequences

In [59]:
!grep ":primary" Fundulus_heteroclitus-3.0.2.105-exon.gff

In [65]:
#Find the complement to the exon track (non-coding sequences)
#Create a BEDfile of IGV
!{bedtoolsDirectory}/complementBed \
-i Fundulus_heteroclitus-3.0.2.105-exon.gff \
-g mummichog.chrom.length \
> Fundulus_heteroclitus-3.0.2.105-nonCDS.bed

***** ERROR: illegal number ":primary". Exiting...


In [52]:
!head Fundulus_heteroclitus-3.0.2.105-nonCDS.bed
!wc -l Fundulus_heteroclitus-3.0.2.105-nonCDS.bed

       0 Fundulus_heteroclitus-3.0.2.105-nonCDS.bed


### 2g. Intron

In [18]:
#Find the intersection between the non-coding sequences and genes (introns)
!{bedtoolsDirectory}/intersectBed \
-a Fundulus_heteroclitus-3.0.2.105-nonCDS.bed \
-b Fundulus_heteroclitus-3.0.2.105-gene.gff -sorted \
> Fundulus_heteroclitus-3.0.2.105-intron.bed

In [19]:
!head Fundulus_heteroclitus-3.0.2.105-intron.bed
!wc -l Fundulus_heteroclitus-3.0.2.105-intron.bed

NC_035780.1	13603	14236
NC_035780.1	14290	14556
NC_035780.1	29073	30523
NC_035780.1	31557	31735
NC_035780.1	31887	31976
NC_035780.1	32565	32958
NC_035780.1	44358	45912
NC_035780.1	46506	64122
NC_035780.1	64334	66868
NC_035780.1	85777	88422
  311341 C_virginica-3.0-intron.bed


### 2i. Flanking regions (1 kb)

#### All flanks

In [22]:
#Create 1 kb flanking regions
#Subtract existing genes from artificial flanks
!{bedtoolsDirectory}/flankBed \
-i C_virginica-3.0-gene.gff \
-g C_virginica-3.0-sequence-lengths.txt \
-b 1000 \
| {bedtoolsDirectory}/subtractBed \
-a - \
-b C_virginica-3.0-gene.gff \
> C_virginica-3.0-flanks.gff

In [23]:
!head C_virginica-3.0-flanks.gff
!wc -l C_virginica-3.0-flanks.gff

NC_035780.1	Gnomon	gene	12578	13577	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	gene	14595	15594	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	gene	27961	28960	.	+	.	ID=gene-LOC111126949;Dbxref=GeneID:111126949;Name=LOC111126949;gbkey=Gene;gene=LOC111126949;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	33325	34324	.	+	.	ID=gene-LOC111126949;Dbxref=GeneID:111126949;Name=LOC111126949;gbkey=Gene;gene=LOC111126949;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	42111	43110	.	-	.	ID=gene-LOC111110729;Dbxref=GeneID:111110729;Name=LOC111110729;gbkey=Gene;gene=LOC111110729;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	66898	67897	.	-	.	ID=gene-LOC111110729;Dbxref=GeneID:111110729;Name=LOC111110729;gbkey=Gene;gene=LOC111110729;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	84606	85605	.	-	.	ID

#### Upstream flanks

In [24]:
#Create 1 kb upstream flanking regions (-l) based on strand (-s)
#Subtract existing genes from artificial flanks
!{bedtoolsDirectory}/flankBed \
-i C_virginica-3.0-gene.gff \
-g C_virginica-3.0-sequence-lengths.txt \
-l 1000 \
-r 0 \
-s \
| {bedtoolsDirectory}/subtractBed \
-a - \
-b C_virginica-3.0-gene.gff \
> C_virginica-3.0-upstream.gff

In [25]:
!head C_virginica-3.0-upstream.gff
!wc -l C_virginica-3.0-upstream.gff

NC_035780.1	Gnomon	gene	12578	13577	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	gene	27961	28960	.	+	.	ID=gene-LOC111126949;Dbxref=GeneID:111126949;Name=LOC111126949;gbkey=Gene;gene=LOC111126949;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	66898	67897	.	-	.	ID=gene-LOC111110729;Dbxref=GeneID:111110729;Name=LOC111110729;gbkey=Gene;gene=LOC111110729;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	95255	96254	.	-	.	ID=gene-LOC111112434;Dbxref=GeneID:111112434;Name=LOC111112434;gbkey=Gene;gene=LOC111112434;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	98840	99839	.	+	.	ID=gene-LOC111120752;Dbxref=GeneID:111120752;Name=LOC111120752;gbkey=Gene;gene=LOC111120752;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	110078	111077	.	-	.	ID=gene-LOC111128944;Dbxref=GeneID:111128944;Name=LOC111128944;gbkey=Gene;gene=LOC111128944;gene_biotype=protein_coding;partial=true;start_range=.,108305


#### Downstream flanks

In [26]:
#Create 1 kb upstream flanking regions (-l) based on strand (-s)
#Subtract existing genes from artificial flanks
!{bedtoolsDirectory}/flankBed \
-i C_virginica-3.0-gene.gff \
-g C_virginica-3.0-sequence-lengths.txt \
-l 0 \
-r 1000 \
-s \
| {bedtoolsDirectory}/subtractBed \
-a - \
-b C_virginica-3.0-gene.gff \
> C_virginica-3.0-downstream.gff

In [27]:
!head C_virginica-3.0-downstream.gff
!wc -l C_virginica-3.0-downstream.gff

NC_035780.1	Gnomon	gene	14595	15594	.	+	.	ID=gene-LOC111116054;Dbxref=GeneID:111116054;Name=LOC111116054;gbkey=Gene;gene=LOC111116054;gene_biotype=lncRNA
NC_035780.1	Gnomon	gene	33325	34324	.	+	.	ID=gene-LOC111126949;Dbxref=GeneID:111126949;Name=LOC111126949;gbkey=Gene;gene=LOC111126949;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	42111	43110	.	-	.	ID=gene-LOC111110729;Dbxref=GeneID:111110729;Name=LOC111110729;gbkey=Gene;gene=LOC111110729;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	84606	85605	.	-	.	ID=gene-LOC111112434;Dbxref=GeneID:111112434;Name=LOC111112434;gbkey=Gene;gene=LOC111112434;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	106461	107460	.	+	.	ID=gene-LOC111120752;Dbxref=GeneID:111120752;Name=LOC111120752;gbkey=Gene;gene=LOC111120752;gene_biotype=protein_coding
NC_035780.1	Gnomon	gene	107305	108304	.	-	.	ID=gene-LOC111128944;Dbxref=GeneID:111128944;Name=LOC111128944;gbkey=Gene;gene=LOC111128944;gene_biotype=protein_coding;partial=true;start_range=.,10830

### 2j. Intergenic regions

In [28]:
#Find the complement of genes, then subtract flanks to obtain intergenic regions
!{bedtoolsDirectory}/complementBed \
-i C_virginica-3.0-gene.gff -sorted \
-g C_virginica-3.0-sequence-lengths.txt \
| {bedtoolsDirectory}/subtractBed \
-a - \
-b C_virginica-3.0-flanks.gff \
> C_virginica-3.0-intergenic.bed

In [29]:
!head C_virginica-3.0-intergenic.bed
!wc -l C_virginica-3.0-intergenic.bed

NC_035780.1	0	12577
NC_035780.1	15594	27960
NC_035780.1	34324	42110
NC_035780.1	67897	84605
NC_035780.1	96254	98839
NC_035780.1	111077	150858
NC_035780.1	158536	162808
NC_035780.1	184798	189448
NC_035780.1	194594	203242
NC_035780.1	208743	213890
   23949 C_virginica-3.0-intergenic.bed
